<a href="https://colab.research.google.com/github/Vampiracle/PO/blob/main/catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from itertools import combinations
import json

!pip install annoy
from annoy import AnnoyIndex
import matplotlib.pyplot as plt








%matplotlib inline

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [73]:
!ls /content/drive/MyDrive/ML
game_train = pd.read_csv("/content/drive/MyDrive/ML/train.csv")
#game_test = pd.read_csv("/content/drive/MyDrive/ML/test.csv")
game_train1 = pd.read_csv("/content/drive/MyDrive/ML/Donda/X_train.csv")
game_train2 = pd.read_csv("/content/drive/MyDrive/ML/Donda/y_train.csv")

Donda	    kek1       prep3.csv  sample_submission.csv  togeth.npy  versus.npy
heroes.npy  prep1.csv  prep.csv   test.csv		 train.csv


In [74]:
from sklearn import metrics


print(metrics.log_loss(np.array(game_train["target"].values), ans))

9.99200722162641e-16


In [75]:



game_train1["target"] = game_train2["target"]
game_train1 = game_train1.drop("Unnamed: 0", 1)

game_train1.insert(0, "id", game_train["id"])


game_train1 = game_train1.rename(columns={"battle_regime":'X0', "player1_id":'X1', "player1_rate":'X2', "player2_id":'X3', "player2_rate":'X4', "game_duration":'X21'})

In [76]:
game_train1[0:3:]

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,target
0,0,2,672429,1051,480768,1065,54,48,3,92,96,55,5,51,89,10,19,94,70,79,26,38,343,1
1,1,1,806472,1017,97770,1024,82,89,81,33,19,43,10,69,34,32,7,10,90,12,25,35,601,0
2,2,2,414907,994,227065,993,23,53,56,75,93,12,88,63,57,39,88,68,63,87,92,1,677,0


In [77]:
game_train[0:3:]

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,target
0,0,2,908749,954,480270,935,71,79,53,89,15,1,75,80,51,100,87,3,61,48,43,79,329,0
1,1,1,438444,2162,486685,2154,32,39,48,49,100,18,57,26,39,83,91,6,57,94,15,89,581,1
2,2,1,596915,2066,711059,2081,43,21,58,94,88,93,16,61,7,52,37,50,69,60,89,10,548,0


In [78]:
game_train = game_train1

In [79]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 50

In [80]:

pd.options.display.max_columns = 200
pd.options.display.max_rows = 50
for i in range(5, 13):
    game_train = game_train.rename(columns={'X' + str(i):'p1' + 'unit' + str(i - 4)})
for i in range(13, 21):
    game_train = game_train.rename(columns={'X' + str(i):'p2' + 'unit' + str(i - 12)})

In [81]:
game_train = game_train.rename(columns={'X0':'match_type', 'X1':'id1', 'X2':'p1_mmr', 'X3':'id2', 'X4':'p2_mmr', 'X21':'time'})

In [82]:
game_train[:100:]

,id,match_type,id1,p1_mmr,id2,p2_mmr,p1unit1,p1unit2,p1unit3,p1unit4,p1unit5,p1unit6,p1unit7,p1unit8,p2unit1,p2unit2,p2unit3,p2unit4,p2unit5,p2unit6,p2unit7,p2unit8,time,target
0,0,2,672429,1051,480768,1065,54,48,3,92,96,55,5,51,89,10,19,94,70,79,26,38,343,1
1,1,1,806472,1017,97770,1024,82,89,81,33,19,43,10,69,34,32,7,10,90,12,25,35,601,0
2,2,2,414907,994,227065,993,23,53,56,75,93,12,88,63,57,39,88,68,63,87,92,1,677,0
3,3,1,689272,844,683879,853,44,16,80,0,60,26,65,11,43,44,39,89,80,60,78,66,194,0
4,4,1,995460,2189,914464,2177,58,48,18,81,54,96,51,60,26,55,48,21,43,81,72,62,206,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1,756976,1148,377696,1166,51,5,96,3,92,79,48,72,80,32,70,50,20,40,0,64,427,0
96,96,2,1076382,1654,533414,1646,18,48,13,81,65,61,60,58,87,63,80,57,68,56,100,53,626,1
97,97,1,510470,1564,956911,1551,45,91,15,3,79,10,94,24,14,29,64,55,48,24,26,30,590,0
98,98,2,818824,1289,903400,1295,28,86,35,78,94,69,15,10,26,55,48,62,72,43,21,81,666,0


In [83]:
game_train[4151:4152:]


,id,match_type,id1,p1_mmr,id2,p2_mmr,p1unit1,p1unit2,p1unit3,p1unit4,p1unit5,p1unit6,p1unit7,p1unit8,p2unit1,p2unit2,p2unit3,p2unit4,p2unit5,p2unit6,p2unit7,p2unit8,time,target
4151,4151,1,1119445,2398,691337,2394,35,63,88,25,53,100,26,71,16,80,81,22,60,45,44,10,552,0


In [84]:
q = (game_train["p1unit1"].unique())
q = np.sort(q)
print(q)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100]


In [85]:
print(len(game_train["id1"].unique()))

639743


In [86]:
heroes_wrs = [[0] * 2 for i in range(101)]
print(heroes_wrs)
vs_wrs = [[[0 for i in range(2)] for j in range(101)] for k in range(101)] # [i][j] - winrate i-th VS j-th ([0] - wins, [1] - matches) 
together_wrs = [[[0 for i in range(2)] for j in range(101)] for k in range(101)] # [i][j] - winrate i-th VS j-th ([0] - wins, [1] - matches) 



[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]


In [87]:
for index, row in game_train.iterrows():
  t1 = row["p1unit1":"p2unit1":].values
  t2 = row["p2unit1":"time":].values
  flag = row["target"]
  if flag == 1:
    for i in range(8):
      if t1[i] not in t2:
        heroes_wrs[t1[i]][0] += 1
        heroes_wrs[t1[i]][1] += 1
      if t2[i] not in t1:
        heroes_wrs[t2[i]][1] += 1
    

    for i in range(8):
      for j in range(8):
        vs_wrs[t1[i]][t2[j]][0] += 1
        vs_wrs[t1[i]][t2[j]][1] += 1

        vs_wrs[t2[i]][t1[j]][1] += 1
    
    for i in range (8):
      for j in range (8):
        
        together_wrs[t1[i]][t1[j]][0] += 1
        together_wrs[t1[i]][t1[j]][1] += 1
        
        together_wrs[t2[i]][t2[j]][1] += 1


  if flag == 0:
    for i in range(8):
      if t2[i] not in t1:
        heroes_wrs[t2[i]][0] += 1
        heroes_wrs[t2[i]][1] += 1
      if t1[i] not in t2:
        heroes_wrs[t1[i]][1] += 1

    for i in range(8):
      for j in range(8):
        vs_wrs[t2[i]][t1[j]][0] += 1
        vs_wrs[t2[i]][t1[j]][1] += 1

        vs_wrs[t1[i]][t2[j]][1] += 1

    for i in range (8):
      for j in range (8):
        
        together_wrs[t2[i]][t2[j]][0] += 1
        together_wrs[t2[i]][t2[j]][1] += 1
        
        together_wrs[t1[i]][t1[j]][1] += 1
  if index % 10000 == 0:
    print(index)
  

      
  
  




0


KeyboardInterrupt: ignored

In [ ]:

print(heroes_wrs[0][0]/heroes_wrs[0][1])


In [ ]:
print(lol1 / lol2)


In [ ]:

print(heroes_wrs[0][0]/heroes_wrs[0][1])
print(vs_wrs)
print(together_wrs)
kek1 = np.array(heroes_wrs)
kek2 = np.array(vs_wrs)
kek3 = np.array(together_wrs)

#np.save("/content/drive/MyDrive/ML/heroes", kek1)
#np.save('/content/drive/MyDrive/ML/versus', kek2)
#np.save('/content/drive/MyDrive/ML/togeth', kek3)




In [99]:
heroes_wrs = np.load("/content/drive/MyDrive/ML/heroes.npy")
heroes_wrs_end = []
vs_wrs = np.load("/content/drive/MyDrive/ML/versus.npy")
vs_wrs_end = [[0] * 101 for i in range(101)]
together_wrs = np.load("/content/drive/MyDrive/ML/togeth.npy")
together_wrs_end = [[0] * 101 for i in range(101)]
for i in range(101):
  heroes_wrs_end.append(heroes_wrs[i][0]/heroes_wrs[i][1])
for i in range(101):
  for j in range(101):
    vs_wrs_end[i][j] = vs_wrs[i][j][0]/vs_wrs[i][j][1]
for i in range(101):
  for j in range(101):
    together_wrs_end[i][j] = together_wrs[i][j][0]/together_wrs[i][j][1]

  


In [ ]:
from scipy import stats
stats.describe(pd.array(heroes_wrs_end))


In [ ]:
print(heroes_wrs_end) #26 - 49.14

In [ ]:
print(together_wrs_end[20][50])

In [89]:
a = AnnoyIndex(101)
for index, row in game_train.iterrows():
  temp = [0] * 101
  #temp_rev = [0] * 101
  t1 = row["p1unit1":"p2unit1":].values
  t2 = row["p2unit1":"time":].values
  for i in range(8):
    temp[t1[i]] = 1
    temp[t2[i]] = -1
    #temp_rev[t2[i]] = 1
    #temp_rev[t1[i]] = -1
    a.add_item(index, temp)
    #a.add_item(index + len(game_train), temp_rev)
  if index % 1000 == 0:
    print(index, temp)


a.build(20, n_jobs=-1)
a.save("/content/drive/MyDrive/ML/kek2")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


0 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, -1, 0, 1, 0, 0, 0, 0]
1000 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 1, -1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]
2000 [0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3000 [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1, 0, 0, 0, -1

True

In [ ]:
print(a.get_item_vector(2543103))
print(a.get_item_vector(0))

In [ ]:
a = AnnoyIndex(101)
a.load("/content/drive/MyDrive/ML/kek1")
kekus = a.get_nns_by_item(0, 21)
for i in range(1, 21):
  #if kekus[i] > len(game_train):
    #print(kekus[i], game_train.iloc[kekus[i] - len(game_train)])
  #else:
    print(kekus[i], game_train.iloc[kekus[i]])


  

In [ ]:
print(game_train.iloc[0])

In [ ]:
print(game_train["time"])

In [ ]:
azazek = game_train["time"].values
ratings = game_train["match_type"].values
azazek = np.sort(azazek)
ratings = np.sort(ratings)
plt.hist(ratings)
index_for_time1 = 275
index_for_time1 = 550
#plt.hist(azazek)
print(azazek[10:20])
print(game_train["time"][10:20])
utility_train1 = []
utility_train2 = []
utility_train3 = []

In [ ]:
heroes_wrs_end[26]

In [ ]:
lol1 = 0
lol2 = 0
for index, row in game_train.iterrows():
  t1 = row["p1unit1":"p2unit1":].values
  t2 = row["p2unit1":"time":].values
  flag = row["target"]
  
  for i in range(8):
    if t1[i] == 0 and 0 not in t2:
      if flag == 1:
        lol1 += 1
      lol2+=1
  if index % 10000 == 0:
    print(index)  

In [ ]:
print(lol1/lol2)
print(type(game_train["match_type"]))

In [90]:
prediction_train = pd.DataFrame()
prediction_train["match_type"] = game_train["match_type"]
prediction_train["delta_mmr"] = np.subtract(game_train["p1_mmr"], game_train["p2_mmr"])
prediction_train["solo_team"] = game_train["id1"]
solosolo = []
for index, row in game_train.iterrows():
  t1 = row["p1unit1":"p2unit1":].values
  t2 = row["p2unit1":"time":].values
  t1_stats = []
  t2_stats = []
  for i in range(8):
    t1_stats.append(heroes_wrs_end[t1[i]])
    t2_stats.append(heroes_wrs_end[t2[i]])
  t1_stats = pd.array(t1_stats)
  t2_stats = pd.array(t2_stats)
  solosolo.append(t1_stats.mean() - t2_stats.mean())
  if abs(t1_stats.mean() - t2_stats.mean()) > 1:
    print(index)
    break
  
  
  if index % 10000 == 0:
    print(index)
  
  

print(prediction_train)

0


KeyboardInterrupt: ignored

In [ ]:
pam = np.sort(prediction_train["delta_mmr"].values)

#prediction_train["solo_team"] = np.array(solosolo)
prediction_train["target"] = game_train["target"]
print(prediction_train)
count = 0
for index, row in prediction_train.iterrows():
  
  if (row["solo_team"] > 0 and row["target"] == 1) or (row["solo_team"] < 0 and row["target"] == 0):
    count += 1
  
  if index % 10000 == 0:
    print(index)
print(count)


In [ ]:

print(count / 2543000)

In [ ]:
prediction_train["vs_team"] = game_train["id1"]
premas = []
for index, row in game_train.iterrows():
  t1 = row["p1unit1":"p2unit1":].values
  t2 = row["p2unit1":"time":].values
  t1_stats = []
  t2_stats = []
  
  temp = 0
  for i in range(8):
    
    for j in range(8):
      temp += vs_wrs_end[t1[i]][t2[j]]
      
    
    
  
  premas.append(temp / 64)
  
  
  if index % 10000 == 0:
    print(index, temp)
    

In [ ]:
prediction_train["vs_team"] = np.array(premas)
count = 0
for index, row in prediction_train.iterrows():
  
  if (row["vs_team"] > 0.5 and row["target"] == 1) or (row["vs_team"] < 0.5 and row["target"] == 0):
    count += 1
  
  if index % 10000 == 0:
    print(index, count)
print(count)

In [ ]:
print(count / 2543103)
prediction_train.to_csv("/content/drive/MyDrive/ML/prep.csv")

In [ ]:
prediction_train["together_team"] = game_train["id1"]
premas_end = []
for index, row in game_train.iterrows():
  t1 = row["p1unit1":"p2unit1":].values
  t2 = row["p2unit1":"time":].values
  t1_stats = []
  t2_stats = []
  premas = []
  
  for i in range(8):
    for j in range(i + 1, 8):
      premas.append(together_wrs_end[t1[i]][t1[j]] - together_wrs_end[t2[i]][t2[j]])
  premas_end.append(np.array(premas).mean())
  
  
  if index % 10000 == 0:
  
    print(index)
    
  
  
  

In [ ]:
prediction_train["together_team"] = np.array(premas_end)
count = 0
for index, row in prediction_train.iterrows():
  
  if (row["together_team"] > 0 and row["target"] == 1) or (row["together_team"] < 0 and row["target"] == 0):
    count += 1
  
  if index % 10000 == 0:
    print(index, count)
print(count)

In [ ]:
print(count / 2543103)

In [ ]:
print(prediction_train)

In [92]:
prediction_train["knn"] = game_train["id1"]
premas = []
for index, row in game_train.iterrows():
  temp = []
  kekus = a.get_nns_by_item(index, 21)
  for i in range(1, 21):
      
      temp.append(game_train.iloc[kekus[i]]["target"])
  premas.append(np.array(temp).mean())
  
  if index % 10000 == 0:
  
    print(index)
prediction_train["knn"] = np.array(premas)
    

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [ ]:
prediction_train["knn"] = np.array(premas)
prediction_train.to_csv("/content/drive/MyDrive/ML/prep3.csv")  

In [98]:
print(metrics.log_loss(np.array(game_train["target"].values), np.array(prediction_train["knn"].values)))

0.6757131529183567


In [97]:

print(game_train["target"])

0          1
1          0
2          0
3          0
4          0
          ..
2543098    0
2543099    1
2543100    1
2543101    1
2543102    1
Name: target, Length: 2543103, dtype: int64


In [ ]:
print(np.count_nonzero(np.array(game_train["target"].values) == 0))

In [ ]:
kek = pd.read_csv("/content/drive/MyDrive/ML/prep1.csv")

In [ ]:
print(len(kek))


In [102]:
def remake(game_train1, game_train2, heroes_wrs_end, vs_wrs_end, together_wrs_end, a, game_train):
  game_train = pd.read_csv("/content/drive/MyDrive/ML/train.csv")
  game_train1["target"] = game_train2["target"]
  game_train1 = game_train1.drop("Unnamed: 0", 1)
  game_train1.insert(0, "id", game_train["id"])
  game_train1 = game_train1.rename(columns={"battle_regime":'X0', "player1_id":'X1', "player1_rate":'X2', "player2_id":'X3', "player2_rate":'X4', "game_duration":'X21'})
  game_train = game_train1
  for i in range(5, 13):
    game_train = game_train.rename(columns={'X' + str(i):'p1' + 'unit' + str(i - 4)})
  for i in range(13, 21):
    game_train = game_train.rename(columns={'X' + str(i):'p2' + 'unit' + str(i - 12)})
  game_train = game_train.rename(columns={'X0':'match_type', 'X1':'id1', 'X2':'p1_mmr', 'X3':'id2', 'X4':'p2_mmr', 'X21':'time'})
  
  
  prediction_train = pd.DataFrame()
  prediction_train["match_type"] = game_train["match_type"]
  prediction_train["delta_mmr"] = np.subtract(game_train["p1_mmr"], game_train["p2_mmr"])
  prediction_train["solo_team"] = game_train["id1"]
  solosolo = []
  for index, row in game_train.iterrows():
    t1 = row["p1unit1":"p2unit1":].values
    t2 = row["p2unit1":"time":].values
    t1_stats = []
    t2_stats = []
    for i in range(8):
      t1_stats.append(heroes_wrs_end[t1[i]])
      t2_stats.append(heroes_wrs_end[t2[i]])
    t1_stats = pd.array(t1_stats)
    t2_stats = pd.array(t2_stats)
    solosolo.append(t1_stats.mean() - t2_stats.mean())
  
  if index % 10000 == 0:
    print(index)
  prediction_train["solo_team"] = np.array(solosolo)
  prediction_train["target"] = game_train["target"]
  

  prediction_train["vs_team"] = game_train["id1"]
  premas = []
  for index, row in game_train.iterrows():
    t1 = row["p1unit1":"p2unit1":].values
    t2 = row["p2unit1":"time":].values
    t1_stats = []
    t2_stats = []
    temp = 0
    for i in range(8):
    
      for j in range(8):
        temp += vs_wrs_end[t1[i]][t2[j]]
    premas.append(temp / 64)
    if index % 10000 == 0:
      print(index, temp)
  prediction_train["vs_team"] = np.array(premas)
  

  prediction_train["together_team"] = game_train["id1"]
  premas_end = []
  for index, row in game_train.iterrows():
    t1 = row["p1unit1":"p2unit1":].values
    t2 = row["p2unit1":"time":].values
    t1_stats = []
    t2_stats = []
    premas = []
  
    for i in range(8):
      for j in range(i + 1, 8):
        premas.append(together_wrs_end[t1[i]][t1[j]] - together_wrs_end[t2[i]][t2[j]])
    premas_end.append(np.array(premas).mean())
  
  
    if index % 10000 == 0:
  
      print(index)
  prediction_train["together_team"] = np.array(premas_end)


  prediction_train["knn"] = game_train["id1"]
  premas = []
  for index, row in game_train.iterrows():
    temp = []
    kekus = a.get_nns_by_item(index, 21)
    for i in range(1, 21):
      
        temp.append(game_train.iloc[kekus[i]]["target"])
    premas.append(np.array(temp).mean())
  
    if index % 10000 == 0:
  
      print(index)
  prediction_train["knn"] = np.array(premas)
  return prediction_train

In [103]:
df1 = pd.read_csv("/content/drive/MyDrive/ML/Donda/X_train.csv")
df2 = pd.read_csv("/content/drive/MyDrive/ML/Donda/y_train.csv")
df3 = pd.read_csv("/content/drive/MyDrive/ML/Donda/X_val.csv")
df4 = pd.read_csv("/content/drive/MyDrive/ML/Donda/y_val.csv")
df5 = pd.read_csv("/content/drive/MyDrive/ML/Donda/X_test.csv")
df6 = pd.read_csv("/content/drive/MyDrive/ML/Donda/y_test.csv")
df11 = remake(df1, df2, heroes_wrs_end, vs_wrs_end, together_wrs_end, a, game_train)
df11.to_csv("/content/drive/MyDrive/ML/11.csv") 
df22 = remake(df3, df4, heroes_wrs_end, vs_wrs_end, together_wrs_end, a, game_train)
df22.to_csv("/content/drive/MyDrive/ML/22.csv") 
df33 = remake(df5, df6, heroes_wrs_end, vs_wrs_end, together_wrs_end, a, game_train)
df33.to_csv("/content/drive/MyDrive/ML/33.csv") 


0 32.33432526689629
10000 31.641373921036134
20000 32.39071744008372
30000 31.75743165459084
40000 32.11686090088818
50000 31.559733271981063
60000 32.53873925649766
70000 31.999999999999996
80000 32.1261096348043
90000 30.579586067918058
100000 32.08274607401507
110000 31.965928801289714
120000 31.173102884380015
130000 32.19096743023165
140000 32.90071478840768
150000 32.512722454669436
160000 31.79803738455157
170000 31.689009008867906
180000 32.08896266642408
190000 31.679221759761305
200000 31.337327484761396
210000 30.902556431410137
220000 32.074856670830854
230000 31.649218042547165
240000 31.166350795519612
250000 31.900278877475174
260000 30.700092396663514
270000 31.590032702743425
280000 32.04249555792413
290000 32.473366472341546
300000 32.02779295575973
310000 31.9786820779282
320000 34.05540247254621
330000 31.957475990516457
340000 32.88959164419666
350000 33.15037937138223
360000 32.42048990135394
370000 31.918698631677547
380000 32.636639304153256
390000 32.0685758607

IndexError: ignored